In [1]:
import sys  
sys.path.insert(0, '/home/iftekhar/myworkplace/AI-system/retrieval_Model/Page_Ranking_Experiment/methods_collection/')
import ranking_help_methods 
import make_question as question_maker
import vectorization
import sequence_handler
import corpus_handling_methods
import pandas as pd
import ranking
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
TFIDF_vectorizer = TfidfVectorizer()

In [2]:
dataset = pd.read_csv("../../../Helpers/Title_link_merged_corpus.csv")
dataset = dataset.iloc[:,2:]
dataset = dataset.rename(columns={"Article": "Data"})
dataset.Data = dataset.Data.apply(lambda x: corpus_handling_methods.clean_text(x))
dataset.head()

,Data,Summary,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,MobiControl v14 Manual – MobiControl v14 Manual,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,Apple製品 – MobiControl v14 Manual ios overall,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,Android端末 – MobiControl v14 Manual plus overall,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,Windows PC – MobiControl v14 Manual wpc overall,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,Windows Embedded – MobiControl v14 Manual wm o...,4


In [3]:
split_corpus = corpus_handling_methods.corpus_split(dataset, 8)
split_corpus.head()

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,設定 順序 MobiControl 設定 順序 行い 端末 OS,0
2,対象 オンプレミス MobiControl サーバ のみ 必要 な 設定,0
3,事項 iOS 端末 macOS コンピュータ 必須 接続 プロファイル,0
4,作成 ADDS Active Directory Domain Service サーバ 間,0


In [4]:
perpage_dataset = corpus_handling_methods.corpus_per_page(split_corpus)
perpage_dataset.head() 

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,4


In [16]:
sample_size = 10
questions_samples = question_maker.question_dataframe_generator_1000(split_corpus, sample_size)
questions_samples.head()

,Question,PageID
0,経由 情報 抜き取り など 不能 ランチャー 設定 どちら,6
1,接続 時間 更新 スケジュール 上手 組み合わせる 必要 どう,76
2,始まっ から 操作 マクロ として 保存 可能 どっち,94
3,そして MobiControl アプリ エージェントアプリ 管理 対象 アプリ 何ができますか,227
4,ADDS 接続 プロファイル 作成 参照 ID プロバイダ どこ,223


In [7]:
vec = vectorization.vector_fit(TFIDF_vectorizer, split_corpus)

In [18]:
sample_count = 0
sum_score = 0
container = []
extra_ranks = 0
for index, col in questions_samples.iterrows():
    print(col['Question'], col['PageID'])
    query = str(col['Question'])
    question_parts = question_maker.making_query_collection(query)
    collector = sequence_handler.sequence_searcher(perpage_dataset, question_parts)
    perpage_sequence_match = sequence_handler.perpage_sequence_match(collector, 
                                        perpage_dataset, split_corpus, query)
    ranks = ranking.crude_ranks(perpage_sequence_match, query, vec)
    filtered_ranks = ranking.filtering_ranks(ranks, perpage_sequence_match, query, vec)
    print(col['PageID'], "<=>", filtered_ranks)
    
    if filtered_ranks[0][1] < 0.75:
        print("Calling Pretrained Model")
        
    page_answers = []
    prediction_scores = []
    for ids, score in filtered_ranks:
        page_answers.append(ids)
        prediction_scores.append(score)
    MRR_score = ranking.mean_reciprocal_rank_score(col['PageID'], page_answers)
    sum_score += MRR_score
    container.append([MRR_score, col['PageID'], page_answers, prediction_scores, col['Question']])
    sample_count += 1
    
result = pd.DataFrame(container, columns=['score', 'actual_answer',
'page_answers', 'prediction_scores', 'query'])
result.to_csv('seq_matcher_TFIDF_Vectorizer_performance.csv')
score = sum_score/sample_count
score



経由 情報 抜き取り など 不能 ランチャー 設定 どちら 6
6 <=> [[8, 0.7251734370266806], [16, 0.6217580686826693], [444, 0.6217580686826693]]
Calling Pretrained Model
接続 時間 更新 スケジュール 上手 組み合わせる 必要 どう 76
76 <=> [[121, 0.41665979998536934], [103, 0.4101396160038463], [404, 0.4016445109388408]]
Calling Pretrained Model
始まっ から 操作 マクロ として 保存 可能 どっち 94
94 <=> [[94, 0.9372327223207164], [240, 0.9372327223207164], [26, 0.7550069449753745]]
そして MobiControl アプリ エージェントアプリ 管理 対象 アプリ 何ができますか 227
227 <=> [[271, 0.607541554715171], [3, 0.5963395400375092], [374, 0.5456305095563778]]
Calling Pretrained Model
ADDS 接続 プロファイル 作成 参照 ID プロバイダ どこ 223
223 <=> [[18, 0.9320593715826337], [223, 0.9320593715826337], [424, 0.9320593715826337]]
端末 登録 端末 完了 端末 詳細 設定 どの 90
90 <=> [[90, 0.8330484188381315], [372, 0.6215383006457162], [28, 0.5811716329850496]]
ダウンロード れ インストール れ アプリカタログルール 編集 アプリ どう 368
368 <=> [[385, 0.43325591155195975], [374, 0.3818256305483953], [440, 0.3818256305483953]]
Calling Pretrained Model
保護 サポート れ いる 保護 保護 れ どっち 13

0.625

In [14]:
filtered_ranks[0][1]

0.975616546550444

In [15]:
filtered_ranks

[[24, 0.975616546550444], [443, 0.975616546550444], [128, 0.4409370517422057]]

In [41]:
[(406, array([0.39501141])), (412, array([0.39501141])), (406, array([0.37375799]))]

[[2, ['団体 組織 そっ 階層 端末 グループ 作成 組織']],
 [3,
  ['コンソール 操作 従来 端末 グループ 戻し 指定 アプリ',
   '移動 隔離 実施 異常 端末 グループ 構成 プロファイル',
   'PC 特別 端末 グループ 自動 移動 せ プロファイル',
   '従来 端末 グループ 戻し 指定 アプリ 再 インストール',
   'グループ から 異常 端末 グループ エンド ポイント 自動',
   '異常 端末 グループ エンド ポイント 自動 移動 隔離',
   '実施 異常 端末 グループ 構成 プロファイル アプリ 割り当て',
   'ポイント 所属 部門 対応 た 端末 グループ から']],
 [6,
  ['移動 端末 グループ によって 適用 構成 プロファイル 詳細',
   'データ 値 IP アドレス によって 所属 端末 グループ',
   'グループ 私物 端末 グループ 作る 考え られ により',
   'ので 端末 移動 端末 グループ 設定 値 従い',
   '設定 異なる ので 端末 移動 端末 グループ 設定',
   'IP アドレス によって 所属 端末 グループ 移動 端末']],
 [7,
  ['Windows Configuration Designer 作成 プロビジョニングパッケージ MobiControl 端末 グループ',
   'ユーザ 間 端末 共用 G 異なる 端末 グループ',
   'グループ 端末 全部 働きかけ しか つ 端末 グループ',
   '表示 たい として 端末 グループ あて メッセージ 送る',
   'イン ポート 所属 端末 グループ 再 配置 可能',
   'なる 表示 同じ 端末 グループ 異なる 端末 OS',
   '月日 MobiControl v v ハイライト 異なる 端末 グループ',
   '全般 端末 所属 端末 グループ に関する CSV ファイル',
   '従って つ 端末 グループ 全 端末 働きかけ 方式',
   'つ 作成 G 異なる 端末 グループ ユーザ 間',
   'まし た 自動 DEP 登録 端末 端末 グループ',
   '端末 共用 端末 グループ 異なる 構成 プロファイ